In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from holodeck2 import utils, physics
from holodeck2.constants import *

In [ ]:
def set_axis_color(ax, color, side='right'):
    ax.tick_params(axis='y', which='both', colors=color)
    ax.yaxis.label.set_color(color)
    ax.spines[side].set_color(color)
    return


In [ ]:
def time_from_dadt(rads, dadt):
    xx = rads[::-1]
    yy = dadt[::-1]
    time = -utils.trapz_loglog(1.0/yy, xx)
    time = np.cumsum(time)
    return time[::-1]


class Phenom:
    rad_crit = 10.0    # [pc]
    lifetime = 1.0      # [Gyr]
    # plaw_inner = -0.5
    plaw_inner = -0.5
    plaw_outer = +2.5

    @classmethod
    def dadt(cls, hamp, rads):
        dadt = (
            hamp *
            np.power(rads/cls.rad_crit, 1.0 - cls.plaw_inner) *
            np.power(1.0 + rads/cls.rad_crit, cls.plaw_inner - cls.plaw_outer)
        )
        return dadt


class Inverse:
    rad_crit = 10.0 * PC    # [pc]
    # lifetime = 1.0      # [Gyr]
    # plaw_inner = -0.5
    # plaw_outer = +2.5
    plaw_inner = -0.5
    plaw_outer = +2.5

    @classmethod
    def dadt_ph(cls, hamp, rads):
        dadt = hamp / (
            np.power(rads/cls.rad_crit, cls.plaw_inner - 1.0) +
            np.power(rads/cls.rad_crit, cls.plaw_outer - 1.0)
        )
        return dadt

    @classmethod
    def _time_integ_ph(cls, hamp, rads):
        rads = np.asarray(rads)
        rc = cls.rad_crit
        pli = cls.plaw_inner
        plo = cls.plaw_outer
        return (rc / hamp) * (np.power(rads/rc, pli)/pli + np.power(rads/rc, cls.plaw_outer)/plo)

    @classmethod
    def time_ph(cls, hamp, rads):
        # time = cls._time_integ_ph(hamp, rads[-1]) - cls._time_integ_ph(hamp, rads[:-1])
        time = cls._time_integ_ph(hamp, rads)
        return (time[-1] - time[:-1])

    @classmethod
    def dadt_gw(cls, m1, m2, rads):
        _GW_DADT_SEP_CONST = (64.0 / 5.0) * np.power(NWTG, 3) / np.power(SPLC, 5)
        return _GW_DADT_SEP_CONST * m1 * m2 * (m1 + m2) / np.power(rads, 3)

    @classmethod
    def hard_for_time(cls, time_target, rad_inner, rad_outer):
        # tinner = cls._time_integ_ph(1.0, rad_inner)
        # touter = cls._time_integ_ph(1.0, rad_outer)
        # time = touter - tinner
        hard = np.diff(cls._time_integ_ph(1.0, [rad_inner, rad_outer]))[0] / time_target
        return hard



In [ ]:

# hard = Phenom()
hard = Inverse()
HARD_AMP = 1.0e20

rads = np.logspace(-1, 3, 100) * PC
dadt = hard.dadt_ph(HARD_AMP, rads)

time = time_from_dadt(rads, dadt)
test = hard.time_ph(HARD_AMP, rads)
# print(f"{time=}")
# print(f"{test=}")

fig, ax = plt.subplots(figsize=[8, 5])
ax.grid(alpha=0.15)

ylim = [dadt.min(), 1e3*dadt.max()]
ax.set(xscale='log', yscale='log', ylabel='da/dt', ylim=ylim)


col = 'C0'
dadt_outer = HARD_AMP * np.power(rads/hard.rad_crit, 1.0 - hard.plaw_outer)
dadt_inner = HARD_AMP * np.power(rads/hard.rad_crit, 1.0 - hard.plaw_inner)
ax.plot(rads, dadt_outer, ls='--', color=col, alpha=0.5)
ax.plot(rads, dadt_inner, ls=':', color=col, alpha=0.5)
ax.plot(rads, dadt, color=col)


col = 'C1'
tw = ax.twinx()
set_axis_color(tw, col)
tw.set(yscale='log', ylabel='Time')

tw.plot(rads[:-1], time[0] - time, color=col)
tw.plot(rads[:-1], test[0] - test, color=col, ls='--', lw=3.0, alpha=0.5)
# tw.plot(rads, rads/dadt, color=col, ls=':', lw=3.0, alpha=0.5)
print(f"{time[0]=:.8e}")

plt.show()


In [ ]:
TIME = 1.0 * GYR
rads = np.logspace(-3, 3, 100) * PC

hard_amp = Inverse.hard_for_time(TIME, rads[0], rads[-1])
# print(f"{hard_amp=}")
time_tot = Inverse.time_ph(hard_amp, [rads[0], rads[-1]])
# print(f"{time/GYR=}")

dadt_ph = Inverse.dadt_ph(hard_amp, rads)
dadt_gw = Inverse.dadt_gw(1e9*MSOL, 1e9*MSOL, rads)
dadt = dadt_ph + dadt_gw
time = time_from_dadt(rads, dadt)
# print(f"{time/GYR=}")
# print(f"{dadt_ph=}")
# print(f"{dadt_gw=}")
# print(f"{dadt=}")

fig, axes = plt.subplots(figsize=[12, 5], ncols=2)

ax = axes[0]
ax.set(xscale='log', yscale='log')
xx = rads / PC
ax.plot(xx, dadt_ph, ls='--', label='ph')
ax.plot(xx, dadt_gw, ls='--', label='gw')
ax.plot(xx, dadt, label='tot')

ax.legend()
plt.show()